# GeoLife Project
## Chapter 1 - Download files
1. Import libraries.
2. Mount google drive.
3. Created function for download zip-file.
4. Unzip and moved file into google drive

In [ ]:
from google.colab import drive
import requests
import re

drive.mount('/content/drive')

def download_file(url="https://download.microsoft.com/"
            "download/F/4/8/F4894AA5-FDBC-481E-9285-D5F8C4C4F039/"
            "Geolife%20Trajectories%201.3.zip"):
    r = requests.get(url, allow_redirects=True)
    filename = "Geolife Trajectories 1.3.zip"

    open(filename, 'wb').write(r.content)

!unzip Geolife\ Trajectories\ 1.3.zip
!mv /content/"Geolife Trajectories 1.3" /content/drive/MyDrive/GeoLifeProject

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
!rm -r spark-3.0.1-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()   